In [1]:
import pandas as pd
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('vader_lexicon')
import spacy
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer, word_tokenize
from nltk.stem import WordNetLemmatizer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
nlp = spacy.load('en_core_web_md')
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
from transformers import pipeline
# Create a summarization pipeline using Hugging Face Transformers library
summarizer = pipeline("summarization")
import torch
from transformers import BertTokenizer, BertModel
from bertopic import BERTopic
import io

[nltk_data] Downloading package punkt to /Users/saiyam/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/saiyam/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/saiyam/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
/Users/saiyam/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [2]:
# Function to store the user input in a variable
def store_input():
    global user_rant
    user_rant = user_input_text.get("1.0", "end-1c")

# Function to save the selected option in the 'disorder' variable
def save_choice():
    global disorder_name
    disorder_name = selected_option.get()

# Create the tkinter window
window = tk.Tk()
window.title("Store User Input in a Variable")

window.geometry("800x600")  # Set the initial window size

# Create a text widget to get the user's input
user_input_label = tk.Label(window, text="Enter your rant:")
user_input_text = tk.Text(window, height=30, width=100)

# Create a button to store the input
store_button = tk.Button(window, text="Store Input", command=store_input)

# Place widgets in the window

# Create a variable to hold the selected option
selected_option = tk.StringVar()



# Create Radiobuttons with 5 options
option1 = tk.Radiobutton(window, text="OCD", variable=selected_option, value="obsessive-compulsive-disorder-ocd")
option2 = tk.Radiobutton(window, text="Anxiety", variable=selected_option, value="anxiety")
option3 = tk.Radiobutton(window, text="Depression", variable=selected_option, value="clinical-depression")
option4 = tk.Radiobutton(window, text="PTSD", variable=selected_option, value="post-traumatic-stress-disorder-ptsd")
option5 = tk.Radiobutton(window, text="Bipolar Disorder", variable=selected_option, value="bipolar-disorder")

# Create a button to save the selected option
save_button = tk.Button(window, text="Save", command=save_choice)

# Place widgets in the window

option1.pack()
option2.pack()
option3.pack()
option4.pack()
option5.pack()
save_button.pack()
user_input_label.pack()
user_input_text.pack()
store_button.pack()

# Initialize the 'disorder' variable
disorder_name = None
# Start the tkinter main loop
window.mainloop()
print("User Rant:", user_rant)

User Rant: Hi guys. I think I have OCD but I don't want to get into the details of it right now. But it's getting to the point where I feel like I need help, or at least to talk to a professional, but I don't know how to. This might sound dumb, but I don't know what to say when I call the doctors, do I tell the receptionist "hey I think I have OCD, can I talk to a doctor?". I'm just struggling on how to approach it. Do I need to even tell the receptionist that I need do see a doctor because I feel like I have OCD or can I just say its for mental health?


In [3]:
df=pd.read_excel('mental_health_data.xlsx')
df.head()

,title,tag,Concern,Suggestion
0,Medication experiences,obsessive-compulsive-disorder-ocd,"Hello. I am a 51 y/o man, suffering from Bipo...",drugs.com says their are no interaction issues.
1,Medication experiences,obsessive-compulsive-disorder-ocd,"Hello. I am a 51 y/o man, suffering from Bipo...",I've been on Cymbalta against my will for 9 ye...
2,Depression/OCD crisis,obsessive-compulsive-disorder-ocd,"Trigger warning‚Ä¶,Having a breakdown of crisi...",I feel that. I was for real having a breakdown...
3,Depression/OCD crisis,obsessive-compulsive-disorder-ocd,"Trigger warning‚Ä¶,Having a breakdown of crisi...","I also have OCD so I get it, I really do"
4,Depression/OCD crisis,obsessive-compulsive-disorder-ocd,"Trigger warning‚Ä¶,Having a breakdown of crisi...",üò•üò•‚ù§üíî


In [4]:
def remove_random_strings(text):
    pattern = r'[^a-zA-Z0-9\s]'
    return re.sub(pattern, '', text)
df['Concern'] = df['Concern'].apply(remove_random_strings)

In [5]:
def remove_stopwords(text):
    words = nltk.word_tokenize(text)
    stop_words = set(stopwords.words("english"))
    filtered_words = [word for word in words if word.lower() not in stop_words]
    filtered_words = [word for word in filtered_words if word.isalnum()]
    return " ".join(filtered_words)

# Assuming df['Concern'] contains your text data
df['Concern'] = df['Concern'].str.lower()
# df['Concern'] = df['Concern'].astype(str).apply(remove_stopwords)
print(df['Concern'])

0        hello  i am a 51 yo man suffering from bipolar...
1        hello  i am a 51 yo man suffering from bipolar...
2        trigger warninghaving a breakdown of crisis i ...
3        trigger warninghaving a breakdown of crisis i ...
4        trigger warninghaving a breakdown of crisis i ...
                               ...                        
29049    i havent been active on here in awhileice just...
29050    i havent been active on here in awhileice just...
29051    i havent been active on here in awhileice just...
29052    i havent been active on here in awhileice just...
29053    i havent been active on here in awhileice just...
Name: Concern, Length: 29054, dtype: object


In [6]:
df_disorder=df[df["tag"]==disorder_name]

In [7]:
df_disorder= df_disorder[['Concern']].drop_duplicates().reset_index(drop=True)
df_disorder['User Concern']=user_rant

In [8]:
# Initialize the VADER sentiment analyzer
def concern_sentiment(df_disorder):
    
    analyzer = SentimentIntensityAnalyzer()

    # Calculate the sentiment score for the single "User Concern" value
    user_concern_sentiment = analyzer.polarity_scores(df_disorder.iloc[0]['User Concern'])['compound']

    # Apply the sentiment score to all rows in the "Concern" column

    df_disorder['Concern_Sentiment'] = df_disorder['Concern'].apply(lambda x: analyzer.polarity_scores(x)['compound'])
    df_disorder['User_Concern_Sentiment'] = user_concern_sentiment

    # Print the updated DataFrame
    return df_disorder

In [9]:
#df_disorder= df_disorder[(df_disorder['Concern_Sentiment']<0.05) & (df_disorder['User_Concern_Sentiment']<0.05)]

In [10]:
def word_vector_similarity(text1, text2):
    doc1 = nlp(text1)
    doc2 = nlp(text2)
    return doc1.similarity(doc2)

# Create a new DataFrame with product name, product review, and similarity score
# similarity_score = pd.DataFrame({
#     'Product Name': df['item'],
#     'Concern': df_disorder['Concern'],
#     'Similarity Score': ''
# })

# this adds the simlarity score of each comment in the concern column with the user input


In [11]:
def suggestions(df_disorder):
    df_disorder['Similarity Score'] = df_disorder['Concern'].apply(word_vector_similarity, text2=user_rant)
    df_disorder=df_disorder[df_disorder['Similarity Score']>0.5].sort_values(by='Similarity Score', ascending=False).reset_index(drop=True)[:10]
    df_sug= df_disorder.merge(df, on='Concern', how='left')[['Concern','User Concern','Similarity Score','Concern_Sentiment','User_Concern_Sentiment','Suggestion']]
    df_sug= df_disorder.merge(df, on='Concern', how='left')[['Concern','User Concern','Similarity Score','Concern_Sentiment','User_Concern_Sentiment','Suggestion']]
    df_sug['Suggestion']= df_sug['Suggestion'].astype(str)

    # Combine rows into one paragraph
    paragraph = ' '.join(df_sug['Suggestion'])

    # Load a pre-trained spaCy model (English)
    nlp = spacy.load('en_core_web_md')
    nlp.max_length = 2000000 # or even higher

    # Split the paragraph into sentences
    doc = nlp(paragraph)
    sentences = [sent.text for sent in doc.sents]
    return sentences, doc, df_disorder

In [12]:
def similarity_matrix_func(df_disorder):
    df_sug = df_disorder.merge(df, on='Concern', how='left')[['Concern', 'User Concern', 'Similarity Score', 'Concern_Sentiment', 'User_Concern_Sentiment', 'Suggestion']]
    df_sug['Suggestion'] = df_sug['Suggestion'].astype(str)
    print(1)
    # Combine rows into one paragraph
    paragraph = ' '.join(df_sug['Suggestion'])
    print(1)

    # Load a pre-trained spaCy model (English)
    nlp = spacy.load('en_core_web_md')
    nlp.max_length = 2000000  # or even higher
    print(1)

    # Split the paragraph into sentences
    doc = nlp(paragraph)
    print(5)

    sentences = [sent.text for sent in doc.sents]
    print(1)

    # Initialize a matrix to store sentence similarity scores
    num_sentences = len(sentences)
    print(num_sentences)

    similarity_matrix = np.zeros((num_sentences, num_sentences))
    print(8)

    # Calculate sentence similarity using cosine similarity
    for i in range(num_sentences):
        for j in range(i, num_sentences):  # Only calculate each pair once
            if i != j:
                vector1 = nlp(sentences[i]).vector
                vector2 = nlp(sentences[j]).vector
                similarity = cosine_similarity([vector1], [vector2])[0][0]
                print(similarity, i , j)
                similarity_matrix[i, j] = similarity
                similarity_matrix[j, i] = similarity  # Symmetric
    print(9)

    # Print the sentence similarity matrix (optional)
    print("Sentence Similarity Matrix:")
    print(similarity_matrix)
    return similarity_matrix,df_sug,sentences

In [13]:
def similarity_values(similarity_matrix,sentences):

    # Sum up the similarity values for each sentence
    sentence_similarity_scores = np.sum(similarity_matrix, axis=1)

    # Create a list of (sentence, score) tuples
    sentence_scores_tuples = [(sentence, score) for sentence, score in zip(sentences, sentence_similarity_scores)]

    # Sort the tuples by score in descending order
    sentence_scores_tuples_sorted = sorted(sentence_scores_tuples, key=lambda x: x[1], reverse=True)

    # Select the top 10 sentences with the highest scores
    top_10_sentences = [sentence for sentence, score in sentence_scores_tuples_sorted[:5]]

    # Print the top 10 sentences
    for i, sentence in enumerate(top_10_sentences, 1):
        print(f"Top {i}: {sentence}")
    return top_10_sentences

In [14]:
def generate_summary(content_to_summarize):
    try:
        # Use the summarization pipeline to generate a summary of predefined content
        summary = summarizer(content_to_summarize, min_length=30, do_sample=False)
        summary_textbox.delete("1.0", "end")  # Clear any previous summary
        summary_textbox.insert("1.0", summary[0]['summary_text'])  # Display the summary
    except Exception as e:
        messagebox.showerror("Error", "An error occurred during summarization.")
    return

In [ ]:
df_disorder= concern_sentiment(df_disorder)
sentences, doc, df_disorder= suggestions(df_disorder)
similarity_matrix,df_sug,sentences= similarity_matrix_func(df_disorder)
top_10_sentences=similarity_values(similarity_matrix,sentences)
content_to_summarize = " ".join(top_10_sentences)
window = tk.Tk()
window.title("Mental Health Summary Generator")
# Create a text widget to display the user's input (dummy input)
user_input_label = tk.Label(window, text="Enter your Rant:")
user_input_textbox = tk.Text(window, height=30, width=100)
user_input_textbox.insert("1.0", user_rant)  # Insert the predefined content
summary_label = tk.Label(window, text="Solution Summary:")
summary_textbox = tk.Text(window, height=30, width=100)
generate_summary(content_to_summarize)
# Place widgets in the window
user_input_label.pack()
user_input_textbox.pack()
#generate_button.pack()
summary_label.pack()
summary_textbox.pack()
# Start the tkinter main loop
window.mainloop()

1
1
1
5
1
174
8
0.6551213 0 1
0.5406779 0 2
0.6025117 0 3
0.65538263 0 4
0.5836867 0 5
0.6609541 0 6
0.6608976 0 7
0.59290034 0 8
0.5670778 0 9
0.6031215 0 10
0.5852486 0 11
0.6090447 0 12
0.69462365 0 13
0.5904101 0 14
0.47646964 0 15
0.6278741 0 16
0.56667334 0 17
0.6299391 0 18
0.4017773 0 19
0.5575836 0 20
0.10620391 0 21
0.5989033 0 22
0.59862614 0 23
0.6009712 0 24
0.59925824 0 25
0.68104416 0 26
0.6143288 0 27
0.5579735 0 28
0.6077218 0 29
0.5949914 0 30
0.6074572 0 31
0.6151147 0 32
0.5885017 0 33
0.55033183 0 34
0.49449724 0 35
0.58385074 0 36
0.609756 0 37
0.14030297 0 38
0.51328605 0 39
0.5499756 0 40
0.6089478 0 41
0.48678663 0 42
0.51442146 0 43
0.47526172 0 44
0.6076013 0 45
0.40704775 0 46
0.4979002 0 47
0.5162668 0 48
0.32530814 0 49
0.5122478 0 50
0.41555503 0 51
0.54902196 0 52
0.4612819 0 53
0.66822964 0 54
0.4074741 0 55
0.48643792 0 56
0.25099945 0 57
0.52061546 0 58
0.519855 0 59
0.56481165 0 60
0.714268 0 61
0.46013495 0 62
0.46548012 0 63
0.47274786 0 64
0.60040

0.8363044 3 10
0.7188268 3 11
0.8654654 3 12
0.8035749 3 13
0.8157145 3 14
0.67687154 3 15
0.79513836 3 16
0.7572714 3 17
0.84909093 3 18
0.56605536 3 19
0.7350774 3 20
-0.07194546 3 21
0.8843833 3 22
0.8273402 3 23
0.8164333 3 24
0.8003073 3 25
0.84895957 3 26
0.7479034 3 27
0.8343384 3 28
0.6778543 3 29
0.8161056 3 30
0.88621604 3 31
0.88345414 3 32
0.6655366 3 33
0.7872735 3 34
0.69487655 3 35
0.87432265 3 36
0.855171 3 37
0.04540364 3 38
0.8152745 3 39
0.89812374 3 40
0.8731713 3 41
0.76856565 3 42
0.83570623 3 43
0.78999424 3 44
0.8685593 3 45
0.743636 3 46
0.7985543 3 47
0.818275 3 48
0.34481144 3 49
0.73749864 3 50
0.6714746 3 51
0.7507612 3 52
0.6751833 3 53
0.80841935 3 54
0.6920552 3 55
0.6575183 3 56
0.22392783 3 57
0.54372376 3 58
0.7541585 3 59
0.8411663 3 60
0.83085394 3 61
0.6826942 3 62
0.5605184 3 63
0.7339942 3 64
0.83796126 3 65
0.8729007 3 66
0.87162316 3 67
0.7223261 3 68
0.8687872 3 69
0.8473195 3 70
0.77193105 3 71
0.68070376 3 72
0.6780636 3 73
0.6448492 3 74
0.

0.8582016 6 32
0.68565124 6 33
0.7976206 6 34
0.77901936 6 35
0.87104994 6 36
0.87499565 6 37
0.04104022 6 38
0.8132664 6 39
0.8202491 6 40
0.8871068 6 41
0.73543155 6 42
0.8236423 6 43
0.7247943 6 44
0.8373602 6 45
0.67175245 6 46
0.78516686 6 47
0.8075669 6 48
0.42179355 6 49
0.77338576 6 50
0.66386473 6 51
0.78665006 6 52
0.65952635 6 53
0.8737422 6 54
0.65241647 6 55
0.6478003 6 56
0.22103241 6 57
0.55470645 6 58
0.737881 6 59
0.83949673 6 60
0.9099904 6 61
0.6790962 6 62
0.570764 6 63
0.7563802 6 64
0.8655449 6 65
0.83353764 6 66
0.82666284 6 67
0.8137759 6 68
0.81923974 6 69
0.7603539 6 70
0.6905385 6 71
0.6194999 6 72
0.5655364 6 73
0.5843155 6 74
0.7732627 6 75
0.66227055 6 76
0.6788509 6 77
0.67804277 6 78
0.7749646 6 79
0.39596558 6 80
0.6346885 6 81
0.47386107 6 82
0.8156765 6 83
0.7350588 6 84
0.7736694 6 85
0.8089233 6 86
0.8394791 6 87
0.7675607 6 88
0.74797654 6 89
0.7477843 6 90
0.36838096 6 91
0.7255955 6 92
0.73640656 6 93
0.79913837 6 94
0.83331084 6 95
0.7597148 6 9

0.71695304 9 59
0.7780315 9 60
0.7874619 9 61
0.72058004 9 62
0.6277569 9 63
0.7909344 9 64
0.7333857 9 65
0.8096301 9 66
0.81662476 9 67
0.7213307 9 68
0.7434125 9 69
0.7753885 9 70
0.6929728 9 71
0.69254994 9 72
0.6378685 9 73
0.5775353 9 74
0.7690681 9 75
0.666856 9 76
0.73718256 9 77
0.7088945 9 78
0.75561976 9 79
0.5315956 9 80
0.6924729 9 81
0.592304 9 82
0.7450746 9 83
0.7478915 9 84
0.6929453 9 85
0.78862643 9 86
0.7735357 9 87
0.7854001 9 88
0.7588078 9 89
0.6696781 9 90
0.36603624 9 91
0.7287717 9 92
0.60113096 9 93
0.7200482 9 94
0.82751256 9 95
0.75963366 9 96
0.7595996 9 97
0.79911166 9 98
0.78520834 9 99
0.75981224 9 100
0.8176995 9 101
0.76303357 9 102
0.7311696 9 103
0.83387923 9 104
0.49043846 9 105
0.7831238 9 106
0.62300086 9 107
0.74712545 9 108
0.61878794 9 109
0.7009111 9 110
0.79135525 9 111
0.78182256 9 112
0.829253 9 113
0.721263 9 114
0.674984 9 115
0.73876834 9 116
0.69547695 9 117
0.80429053 9 118
0.78996706 9 119
0.788326 9 120
0.7502945 9 121
0.7214459 9 1

0.8703668 12 75
0.7379972 12 76
0.705464 12 77
0.7534047 12 78
0.8032081 12 79
0.43508446 12 80
0.6634561 12 81
0.57943296 12 82
0.84715515 12 83
0.8332159 12 84
0.81051815 12 85
0.86113775 12 86
0.88490653 12 87
0.8176874 12 88
0.7166177 12 89
0.7767496 12 90
0.34851953 12 91
0.8087634 12 92
0.6214862 12 93
0.75247383 12 94
0.8211758 12 95
0.83030283 12 96
0.91938853 12 97
0.82930243 12 98
0.83938456 12 99
0.73452425 12 100
0.86156404 12 101
0.7976688 12 102
0.87164986 12 103
0.8619667 12 104
0.46137923 12 105
0.8331242 12 106
0.5729553 12 107
0.7240399 12 108
0.69625986 12 109
0.8443134 12 110
0.8115691 12 111
0.8987297 12 112
0.8767874 12 113
0.7873078 12 114
0.6839081 12 115
0.83946717 12 116
0.6969085 12 117
0.6978884 12 118
0.8608273 12 119
0.8072623 12 120
0.74531245 12 121
0.7800584 12 122
0.6254938 12 123
0.56390786 12 124
0.7451751 12 125
0.8175601 12 126
0.8259082 12 127
0.620409 12 128
0.7662771 12 129
0.7734188 12 130
0.83431727 12 131
0.8106859 12 132
0.8001572 12 133
0.7

...

0.74113536 106 125
0.908615 106 126
0.92278385 106 127
0.8059612 106 128
0.8487011 106 129
0.7398132 106 130
0.91519797 106 131
0.8298263 106 132
0.8525548 106 133
0.749975 106 134
0.69029474 106 135
0.8733436 106 136
0.66231 106 137
0.79133004 106 138
0.60362816 106 139
0.92542356 106 140
0.7621629 106 141
0.6390315 106 142
0.88297457 106 143
0.8690995 106 144
0.88412416 106 145
0.7025051 106 146
0.7143043 106 147
0.7995974 106 148
0.7591888 106 149
0.5940093 106 150
0.86494637 106 151
0.8753511 106 152
0.8411199 106 153
0.76417935 106 154
0.9439291 106 155
0.83448696 106 156
0.91173023 106 157
0.7848681 106 158
0.6848875 106 159
0.84022105 106 160
0.84855187 106 161
0.81711626 106 162
0.67516816 106 163
0.89248407 106 164
0.78057003 106 165
0.7768744 106 166
0.81528085 106 167
0.8785239 106 168
0.822907 106 169
0.8736465 106 170
0.88620704 106 171
0.6999418 106 172
0.0 106 173
0.6344418 107 108
0.5860472 107 109
0.55741084 107 110
0.61342096 107 111
0.55272865 107 112
0.6146219 107 1

0.80312645 113 143
0.88587636 113 144
0.91602933 113 145
0.6556946 113 146
0.656835 113 147
0.7582252 113 148
0.63403946 113 149
0.7402619 113 150
0.77144307 113 151
0.88574326 113 152
0.80615747 113 153
0.81962705 113 154
0.8034387 113 155
0.8267886 113 156
0.84782207 113 157
0.90258867 113 158
0.8138038 113 159
0.70653915 113 160
0.82915163 113 161
0.88122445 113 162
0.7608906 113 163
0.80196923 113 164
0.6666068 113 165
0.7287152 113 166
0.6776191 113 167
0.8400547 113 168
0.84397113 113 169
0.877044 113 170
0.81735057 113 171
0.6348233 113 172
0.0 113 173
0.40118673 114 115
0.8357246 114 116
0.62632376 114 117
0.6767247 114 118
0.69172794 114 119
0.61099845 114 120
0.66851056 114 121
0.57958007 114 122
0.35232076 114 123
0.4176288 114 124
0.7028264 114 125
0.6722588 114 126
0.62379706 114 127
0.38324696 114 128
0.63710594 114 129
0.71077454 114 130
0.6464212 114 131
0.6985335 114 132
0.6282111 114 133
0.6128284 114 134
0.63224083 114 135
0.73631173 114 136
0.76003647 114 137
0.7007

0.7546203 121 156
0.8429663 121 157
0.795359 121 158
0.7315037 121 159
0.8001251 121 160
0.8398517 121 161
0.7648616 121 162
0.6739613 121 163
0.82985795 121 164
0.766438 121 165
0.75838447 121 166
0.77691513 121 167
0.84860724 121 168
0.80289054 121 169
0.8173253 121 170
0.81231374 121 171
0.78495157 121 172
0.0 121 173
0.8624988 122 123
0.77833074 122 124
0.7013606 122 125
0.912186 122 126
0.88181067 122 127
0.8343706 122 128
0.8631837 122 129
0.758701 122 130
0.9270966 122 131
0.80936015 122 132
0.8519813 122 133
0.76870316 122 134
0.6843725 122 135
0.81853694 122 136
0.6173456 122 137
0.7789049 122 138
0.617458 122 139
0.8767665 122 140
0.75782967 122 141
0.6619576 122 142
0.85406846 122 143
0.8351145 122 144
0.8185824 122 145
0.6730122 122 146
0.7198669 122 147
0.81559086 122 148
0.7159105 122 149
0.52259994 122 150
0.86484563 122 151
0.85039556 122 152
0.7787038 122 153
0.669323 122 154
0.87936074 122 155
0.78928834 122 156
0.8593726 122 157
0.7343823 122 158
0.6786758 122 159
0.

0.8886459 131 145
0.68385285 131 146
0.7570769 131 147
0.8488527 131 148
0.70585823 131 149
0.61236256 131 150
0.86900187 131 151
0.8924184 131 152
0.8007439 131 153
0.73647463 131 154
0.8668063 131 155
0.84004223 131 156
0.8787194 131 157
0.8256587 131 158
0.7148168 131 159
0.8805171 131 160
0.84523773 131 161
0.80871695 131 162
0.72393274 131 163
0.8654678 131 164
0.8572414 131 165
0.83115494 131 166
0.85397726 131 167
0.92293537 131 168
0.8988282 131 169
0.8793827 131 170
0.90698534 131 171
0.8282426 131 172
0.0 131 173
0.8626525 132 133
0.76151186 132 134
0.728865 132 135
0.80811685 132 136
0.6640361 132 137
0.88187367 132 138
0.6096761 132 139
0.81687206 132 140
0.8126715 132 141
0.7390578 132 142
0.8457101 132 143
0.8839817 132 144
0.833391 132 145
0.6959028 132 146
0.6993347 132 147
0.8020725 132 148
0.70223606 132 149
0.63292295 132 150
0.8803022 132 151
0.88955796 132 152
0.80174744 132 153
0.6864363 132 154
0.79730284 132 155
0.8379849 132 156
0.8231145 132 157
0.83440965 132

0.86452365 144 153
0.7792204 144 154
0.8547119 144 155
0.8747857 144 156
0.8951397 144 157
0.8976932 144 158
0.82734287 144 159
0.73730934 144 160
0.8765447 144 161
0.90164334 144 162
0.8459419 144 163
0.87715435 144 164
0.7672869 144 165
0.7847978 144 166
0.7751453 144 167
0.8760937 144 168
0.86148024 144 169
0.8976238 144 170
0.8438549 144 171
0.68582666 144 172
0.0 144 173
0.6959925 145 146
0.71446794 145 147
0.7825669 145 148
0.7033385 145 149
0.7776586 145 150
0.8042763 145 151
0.8914327 145 152
0.81990695 145 153
0.79424465 145 154
0.85176635 145 155
0.825922 145 156
0.864619 145 157
0.9113876 145 158
0.7812325 145 159
0.79732287 145 160
0.831965 145 161
0.8721959 145 162
0.7897323 145 163
0.8380381 145 164
0.747651 145 165
0.7636783 145 166
0.7283715 145 167
0.8861981 145 168
0.86109525 145 169
0.91743565 145 170
0.85294926 145 171
0.7181458 145 172
0.0 145 173
0.5095011 146 147
0.7807361 146 148
0.8394999 146 149
0.53841853 146 150
0.7100469 146 151
0.78331774 146 152
0.7945744

In [ ]:
df_ocd= df[df['tag']=='obsessive-compulsive-disorder-ocd'].reset_index(drop=True)
df_ptsd= df[df['tag']=='post-traumatic-stress-disorder-ptsd'].reset_index(drop=True)
df_anxiety= df[df['tag']=='anxiety'].reset_index(drop=True)
df_depression= df[df['tag']=='clinical-depression'].reset_index(drop=True)
df_bipolar= df[df['tag']=='bipolar-disorder'].reset_index(drop=True)

In [ ]:
# select only 6000 tweets
docs = df_ocd['Concern'].to_list()
model = BERTopic(verbose=True)
topics, probabilities = model.fit_transform(docs)
model.get_topic_freq().head(11)
model.visualize_barchart()

In [ ]:
# select only 6000 tweets
docs = df_ptsd['Concern'].to_list()
model = BERTopic(verbose=True)
topics, probabilities = model.fit_transform(docs)
model.get_topic_freq().head(11)
model.visualize_barchart()

In [ ]:
docs = df_anxiety['Concern'].to_list()
model = BERTopic(verbose=True)
topics, probabilities = model.fit_transform(docs)
model.get_topic_freq().head(11)
model.visualize_barchart()

In [ ]:
docs = df_depression['Concern'].to_list()
model = BERTopic(verbose=True)
topics, probabilities = model.fit_transform(docs)
model.get_topic_freq().head(11)
model.visualize_barchart()

In [ ]:
docs = df_bipolar['Concern'].to_list()
model = BERTopic(verbose=True)
topics, probabilities = model.fit_transform(docs)
model.get_topic_freq().head(11)
model.visualize_barchart()

I am anxious. i feel very low. PLease help, what can I do?